In [3]:
import random
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta

# Faker kütüphanesini başlat (Türkçe için)
fake = Faker('tr_TR')

# Benzersiz müşteri isimleri oluşturmak için küme kullan
unique_names = set()

# Rastgele müşteri bilgileri oluştur
def generate_customer():
    while True:
        name = fake.name()
        if name not in unique_names:
            unique_names.add(name)
            break
    location = fake.city()
    device = random.choice(['Apple', 'Android'])
    birthdate = fake.date_of_birth(minimum_age=18, maximum_age=25)
    past_purchases = generate_past_purchases()
    return {
        'konum': location,
        'cihaz': device,
        'isim': name,
        'dogum_tarihi': birthdate.strftime('%Y-%m-%d'),
        'gecmis_harcamalar': past_purchases
    }

# Rastgele geçmiş harcamalar oluştur
def generate_past_purchases():
    purchases = []
    for _ in range(random.randint(1, 10)):
        mekan = fake.company()
        currency = '₺'
        amount = random.randint(50, 500)
        timestamp = fake.date_time_between(start_date='-1y', end_date='now').isoformat()
        purchases.append({
            'mekan': mekan,
            'para_birimi': currency,
            'harcama_miktari': amount,
            'zaman': timestamp
        })
    return purchases

# Rastgele teklif bilgileri oluştur
def generate_offer():
    description = fake.sentence()
    location = fake.address()
    photos = [fake.image_url() for _ in range(random.randint(1, 3))]
    age_limit = random.choice([True, False])
    return {
        'tanim': description,
        'mekan': location,
        'fotograflar': photos,
        'yas_limiti': age_limit
    }

# Rastgele mekan bilgileri oluştur
def generate_venue():
    location = fake.address()
    opening_date = fake.date_between(start_date='-5y', end_date='today')
    name = fake.company()
    return {
        'konum': location,
        'acilma_tarihi': opening_date.strftime('%Y-%m-%d'),
        'adi': name
    }

# Rastgele veri seti oluştur
def generate_dataset(num_customers=1000, num_offers=100, num_venues=50):
    customers = [generate_customer() for _ in range(num_customers)]
    offers = [generate_offer() for _ in range(num_offers)]
    venues = [generate_venue() for _ in range(num_venues)]
    
    # Müşteri ve teklifleri eşleştir
    dataset = []
    for customer in customers:
        for offer in offers:
            # Müşteri yaşı ve teklif yaş limiti kontrolü
            birthdate = datetime.strptime(customer['dogum_tarihi'], '%Y-%m-%d')
            age = (datetime.now() - birthdate).days // 365
            if offer['yas_limiti'] and age < 18:
                continue
            # Rastgele bir mekan seç
            venue = random.choice(venues)
            dataset.append({
                'musteri_konum': customer['konum'],
                'musteri_cihaz': customer['cihaz'],
                'musteri_isim': customer['isim'],
                'musteri_dogum_tarihi': customer['dogum_tarihi'],
                'teklif_tanim': offer['tanim'],
                'teklif_mekan': offer['mekan'],
                'teklif_fotograflar': ', '.join(offer['fotograflar']),  # Liste yerine string
                'teklif_yas_limiti': offer['yas_limiti'],
                'mekan_konum': venue['konum'],
                'mekan_acilma_tarihi': venue['acilma_tarihi'],
                'mekan_adi': venue['adi']
            })
    
    # DataFrame oluştur
    df = pd.DataFrame(dataset)
    
    # Boş verileri kontrol et ve doldur
    df['teklif_fotograflar'] = df['teklif_fotograflar'].fillna('https://dummyimage.com/200x200')  # Boşsa varsayılan fotoğraf
    df['teklif_mekan'] = df['teklif_mekan'].fillna(fake.address())  # Boşsa rastgele adres
    df['mekan_konum'] = df['mekan_konum'].fillna(fake.address())  # Boşsa rastgele adres
    df['mekan_adi'] = df['mekan_adi'].fillna(fake.company())  # Boşsa rastgele şirket adı
    
    # Veri tutarlılığını sağla
    df['teklif_yas_limiti'] = df['teklif_yas_limiti'].astype(bool)  # Yaş limiti bool olarak ayarlandı
    df['musteri_dogum_tarihi'] = pd.to_datetime(df['musteri_dogum_tarihi'])  # Tarih formatını düzelt
    df['mekan_acilma_tarihi'] = pd.to_datetime(df['mekan_acilma_tarihi'])  # Tarih formatını düzelt
    
    return df

# Veri setini oluştur ve CSV olarak kaydet (UTF-8 kodlamasıyla)
dataset = generate_dataset()
dataset.to_csv('musteri_teklif_veri_seti.csv', index=False, encoding='utf-8-sig')

print("Veri seti oluşturuldu ve 'musteri_teklif_veri_seti.csv' olarak kaydedildi.")

Veri seti oluşturuldu ve 'musteri_teklif_veri_seti.csv' olarak kaydedildi.


In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import random

# Flask uygulamasını başlat
app = Flask(__name__)

# Veri setini yükle
df = pd.read_csv('musteri_teklif_veri_seti.csv', encoding='utf-8-sig')

# Veri ön işleme
# Gereksiz sütunları kaldır
df = df.drop(columns=['teklif_fotograflar'])

# Tarih sütunlarını Unix zaman damgasına dönüştür
df['musteri_dogum_tarihi'] = pd.to_datetime(df['musteri_dogum_tarihi']).astype('int64') / 10**9
df['mekan_acilma_tarihi'] = pd.to_datetime(df['mekan_acilma_tarihi']).astype('int64') / 10**9

# Müşteri yaşını hesapla
df['musteri_yas'] = (datetime.now().timestamp() - df['musteri_dogum_tarihi']) / (60 * 60 * 24 * 365)

# Kategorik verileri sayısallaştır (musteri_isim hariç)
label_encoders = {}
categorical_columns = ['musteri_konum', 'musteri_cihaz', 'teklif_mekan', 'mekan_konum', 'mekan_adi']
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Bağımlı ve bağımsız değişkenleri ayır
X = df.drop(columns=['teklif_tanim', 'teklif_yas_limiti', 'musteri_isim'])  # musteri_isim sütununu çıkar
y = df['teklif_yas_limiti']

# Veriyi eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model oluştur ve eğit
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Tahmin yap
y_pred = model.predict(X_test)

# Model performansını değerlendir
print("Model Doğruluk Skoru:", accuracy_score(y_test, y_pred))
print("Sınıflandırma Raporu:\n", classification_report(y_test, y_pred))

# Müşteriye mekan önerisi yap
def mekan_oneri(musteri_yas, musteri_gecmis_mekanlar=None):
    # Müşteri yaşına göre uygun mekanları filtrele
    if musteri_yas < 18:
        return {"error": "18 yaşından küçük müşterilere özel mekan önerilemez."}
    
    # Yaş sınırı olmayan veya müşteri yaşına uygun mekanları seç
    uygun_mekanlar = df[(df['teklif_yas_limiti'] == False) | (df['teklif_yas_limiti'] & (musteri_yas >= 18))]
    
    # Müşterinin geçmiş etkinliklerine göre öneri yap
    if musteri_gecmis_mekanlar:
        benzer_mekanlar = uygun_mekanlar[uygun_mekanlar['mekan_adi'].isin(musteri_gecmis_mekanlar)]
        if not benzer_mekanlar.empty:
            onerilen_mekanlar = benzer_mekanlar[['teklif_mekan', 'mekan_adi']].drop_duplicates()
        else:
            onerilen_mekanlar = uygun_mekanlar[['teklif_mekan', 'mekan_adi']].drop_duplicates().sample(n=5)  # Rastgele 5 mekan öner
    else:
        onerilen_mekanlar = uygun_mekanlar[['teklif_mekan', 'mekan_adi']].drop_duplicates().sample(n=5)  # Rastgele 5 mekan öner
    
    # Sayısallaştırılmış değerleri orijinal metin değerlerine dönüştür
    onerilen_mekanlar['teklif_mekan'] = label_encoders['teklif_mekan'].inverse_transform(onerilen_mekanlar['teklif_mekan'])
    onerilen_mekanlar['mekan_adi'] = label_encoders['mekan_adi'].inverse_transform(onerilen_mekanlar['mekan_adi'])
    
    if onerilen_mekanlar.empty:
        return {"error": "Uygun mekan bulunamadı."}
    else:
        return {
            "musteri_yas": musteri_yas,
            "onerilen_mekanlar": onerilen_mekanlar.to_dict(orient='records')
        }

# API endpoint'i
@app.route('/mekan_oneri', methods=['POST'])
def api_mekan_oneri():
    # JSON verisini al
    data = request.json
    musteri_isim = data.get('musteri_isim')
    musteri_yas = data.get('musteri_yas')
    musteri_gecmis_mekanlar = data.get('musteri_gecmis_mekanlar', [])
    
    # Müşteri ismini veri setinde kontrol et
    if musteri_isim not in df['musteri_isim'].values:
        return jsonify({"error": f"'{musteri_isim}' isimli müşteri veri setinde bulunamadı."}), 404
    
    # Mekan önerisi yap
    sonuc = mekan_oneri(musteri_yas, musteri_gecmis_mekanlar)
    return jsonify(sonuc)

# Flask uygulamasını çalıştır
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

Model Doğruluk Skoru: 0.75545
Sınıflandırma Raporu:
               precision    recall  f1-score   support

       False       0.75      0.81      0.78     10767
        True       0.76      0.69      0.72      9233

    accuracy                           0.76     20000
   macro avg       0.76      0.75      0.75     20000
weighted avg       0.76      0.76      0.75     20000

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Jan/2025 23:38:47] "POST /mekan_oneri HTTP/1.1" 200 -
